In [1]:
import numpy as np
import pandas as pd
import os,sys,inspect
import scipy.stats as stats
import scipy.optimize as opt
from scipy.stats import norm
import scipy.integrate as integrate
from scipy.io import loadmat
import econtools 
import econtools.metrics as mt
import statsmodels.discrete.discrete_model as sm
import matplotlib.pyplot as plt
import itertools as it
import copy
import importlib
import warnings
warnings.filterwarnings('ignore')



# my functions
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
#targetdir = os.path.dirname(currentdir)
targetdir = currentdir

if targetdir not in sys.path:
    sys.path.insert(0,targetdir)
    
from entry import entry_likelihood

importlib.reload(entry_likelihood)

<module 'entry.entry_likelihood' from '/Users/jingyuanwang/GitHub/NU450_HW/450-2-HW1/entry/entry_likelihood.py'>

In [2]:
# ------------------------------------------------------------------------
# NOTE
# ------------------------------------------------------------------------
# Purpose: HW1 Q1 
# 
# ------------------------------------------------------------------------

# path

In [3]:
# 0. Initialize file path -----------------------------------------------
rootpath = '/Users/jingyuanwang/Dropbox/Course/ECON/IO/NU450/NU450_HW/450-2-HW1'
datapath = rootpath + '/' + 'data'
resultpath = rootpath + '/' + 'results'

# read data

In [4]:
# Q1
filename = 'entry.mat'
file = datapath + '/'+ filename
data = loadmat(file)

In [5]:
df = (pd.DataFrame(np.concatenate( 
                              (data['mrkts'],
                               data['costs'],
                               data['firms'], 
                               data['entry']), axis = 1 ))
              .rename(columns = {0:'X_m',1:'N_m',2:'pi_1m',3:'pi_2m', 4:'pi_3m', 
                                 5:'phi_1m', 6:'phi_2m', 7:'phi_3m',
                                 8:'Z_1m', 9:'Z_2m', 10:'Z_3m',
                                 11:'entry_1m', 12:'entry_2m', 13:'entry_3m'}) )

In [6]:
# Q2
filename = 'entry2.mat'
file = datapath + '/'+ filename
data = loadmat(file)

In [7]:
df_2 = (pd.DataFrame(np.concatenate( 
                              (data['mrkts'],
                               data['costs'],
                               data['firms'], 
                               data['entry']), axis = 1 ))
              .rename(columns = {0:'X_m',1:'N_m',2:'N_potential_m', 3:'xi_m',
                                 4:'pi_1m',5:'pi_2m', 6:'pi_3m', 7:'pi_4m', 8:'pi_5m', 9:'pi_6m', 10:'pi_7m',
                                 11:'phi_1m', 12:'phi_2m', 13:'phi_3m', 14:'phi_4m', 15:'phi_5m', 16:'phi_6m', 17:'phi_7m',
                                 18:'Z_1m', 19:'Z_2m', 20:'Z_3m', 21:'Z_4m', 12:'Z_5m', 23:'Z_6m', 24:'Z_7m',
                                 25:'entry_1m', 26:'entry_2m', 27:'entry_3m',
                                 28:'entry_4m', 29:'entry_5m', 30:'entry_6m', 31:'entry_7m'}) 
     )

# Q1

In [186]:
importlib.reload(entry_likelihood)

<module 'entry.entry_likelihood' from '/Users/jingyuanwang/GitHub/NU450_HW/450-2-HW1/entry/entry_likelihood.py'>

In [187]:
markets = entry_likelihood.entry_likelihood(M=100, F=3, df=df, n_sample=8000 )

In [188]:
def obj(parameter, order=None):
    
    mu = parameter[0]
    sigma = parameter[1]
    likelihood = markets.get_likelihood(alpha=1, beta=1, delta=1, mu=mu, sigma=sigma, order = order)
    
    return -likelihood

In [189]:
initial = np.array([1,2])
print(obj(initial))
print(obj(initial,order = 'lowestfirst'))

272.0115342856334
274.32620230297454


In [190]:
initial = np.array([2,1])
results0 = opt.minimize(obj, initial, args = None, method = 'Nelder-Mead')
results1 = opt.minimize(obj, initial, args = 'lowestfirst', method = 'Nelder-Mead')

In [192]:
results0

 final_simplex: (array([[3.45892728, 1.85059432],
       [3.45893264, 1.85059825],
       [3.45892996, 1.85058239]]), array([158.64356207, 158.64357202, 158.64365922]))
           fun: 158.64356206778672
       message: 'Optimization terminated successfully.'
          nfev: 107
           nit: 50
        status: 0
       success: True
             x: array([3.45892728, 1.85059432])

In [191]:
results1

 final_simplex: (array([[3.45444201, 1.89487006],
       [3.45440867, 1.89490126],
       [3.4543882 , 1.89490255]]), array([158.64170297, 158.64174103, 158.64174103]))
           fun: 158.64170297107194
       message: 'Optimization terminated successfully.'
          nfev: 87
           nit: 42
        status: 0
       success: True
             x: array([3.45444201, 1.89487006])

# Appendix & draft

In [14]:
entry, phi_fm = markets._simul_entry_order_lowestfirst(alpha=1,  mu=2, sigma=1)

In [19]:
# the first firm that enters
entry * phi_fm 

array([[ 1.45244468,  1.93572603,  0.97129735, ...,  1.9823687 ,
        -0.26262194,  2.06851951],
       [ 0.32683075,  1.63056338,  0.80215822, ...,  2.36570837,
         2.85263776,  0.97864968],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.33171813,  1.81499949,  0.8505708 , ...,  1.86164215,
        -0.38334849,  1.94779296],
       [ 1.91020084,  3.21393347,  2.38552831, ...,  3.94907846,
         4.43600785,  2.56201977],
       [ 3.10259413,  3.34710527,  2.19360224, ...,  3.12697384,
         1.78593717,  1.90124121]])

In [89]:
simul_enter, simul_notenter, simul_order, entry = markets.get_likelihood(alpha=1, 
                                                                         beta=1, 
                                                                         delta=1, 
                                                                         mu=2, 
                                                                         sigma=1,
                                                                        order = 'lowestfirst')

In [93]:
simul_notenter.shape

(300, 10000)

In [128]:
prob = np.sum((entry*simul_enter) + (1-entry)*simul_notenter , axis = 1)/10000
likelihood = np.sum( np.log(prob) )
likelihood

-310.46904612284715

-308.93091414696534

In [144]:
np.product(prob[0:3])

0.06337464

In [140]:
prob2

array([6.340e-02, 8.000e-04, 4.810e-02, 7.890e-01, 2.000e-03, 6.790e-02,
       1.000e-03, 2.929e-01, 8.850e-02, 3.805e-01, 3.340e-02, 1.600e-03,
       1.601e-01, 6.830e-02, 1.800e-03, 4.459e-01, 9.996e-01, 6.380e-02,
       5.110e-02, 1.729e-01, 7.894e-01, 1.200e-02, 7.333e-01, 6.176e-01,
       9.973e-01, 1.010e-02, 5.860e-02, 1.436e-01, 1.778e-01, 9.857e-01,
       2.385e-01, 1.600e-03, 5.110e-02, 1.200e-03, 1.940e-02, 1.840e-02,
       9.370e-02, 1.620e-02, 9.979e-01, 1.420e-01, 4.400e-03, 1.885e-01,
       8.000e-03, 2.476e-01, 1.953e-01, 4.200e-03, 5.100e-03, 1.100e-02,
       3.000e-03, 6.500e-03, 1.320e-02, 1.900e-02, 1.500e-03, 4.080e-02,
       4.270e-02, 5.400e-03, 2.430e-02, 3.200e-03, 1.464e-01, 1.036e-01,
       2.680e-02, 5.500e-03, 1.670e-02, 9.660e-02, 2.400e-01, 9.983e-01,
       6.000e-03, 5.180e-02, 1.066e-01, 1.510e-02, 3.000e-03, 2.150e-02,
       1.000e-04, 7.190e-02, 3.882e-01, 6.940e-01, 1.346e-01, 1.359e-01,
       9.759e-01, 3.290e-02, 2.700e-03, 1.909e-01, 

In [309]:
(simul_enter, simul_notenter) = markets._simul_entry_outcome( alpha=1, beta=1, delta=1, mu=2, sigma=1)

In [310]:
simul_enter

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True, False,  True, ..., False,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True, False,  True],
       [ True, False,  True, ...,  True,  True,  True]])

In [311]:
simul_notenter

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False,  True, False],
       [False, False, False, ..., False, False, False]])

In [341]:
prob = np.sum((entry*simul_enter) + (1-entry)*simul_notenter , axis = 1)/10000

In [343]:
prob[prob<=precision] = precision
np.sum( np.log(prob) )

-310.31318557158556

In [357]:
prob.shape

(300,)